# セマンティックカーネル

このコードサンプルでは、[Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AIフレームワークを使用して、基本的なエージェントを作成します。

このサンプルの目的は、後のコードサンプルでさまざまなエージェントパターンを実装する際に使用する手順を示すことです。


## 必要なPythonパッケージをインポート


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## クライアントの作成

このサンプルでは、LLMにアクセスするために[GitHub Models](https://aka.ms/ai-agents-beginners/github-models)を使用します。

`ai_model_id`は`gpt-4o-mini`として定義されています。GitHub Modelsマーケットプレイスで利用可能な別のモデルに変更して、異なる結果を確認してみてください。

GitHub Modelsの`base_url`に使用される`Azure Inference SDK`を利用するために、Semantic Kernel内で`OpenAIChatCompletion`コネクタを使用します。他にも、Semantic Kernelを使用して他のモデルプロバイダーに対応する[利用可能なコネクタ](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion)があります。


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## エージェントの作成

以下では、`TravelAgent`というエージェントを作成します。

この例では非常にシンプルな指示を使用しています。指示を変更することで、エージェントがどのように異なる反応をするかを確認できます。


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## エージェントの実行

次に、`ChatHistoryAgentThread` 型のスレッドを定義してエージェントを実行します。必要なシステムメッセージは、エージェントの `invoke_stream` メソッドの `messages` キーワード引数に渡します。

これらを定義した後、ユーザーがエージェントに送信する内容となる `user_inputs` を作成します。この例では、このメッセージを `Plan me a sunny vacation` に設定しています。

このメッセージを変更して、エージェントがどのように異なる応答をするか試してみてください。


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があることをご承知ください。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当社は責任を負いません。
